In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [41]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
#Droping features which will not contribute to model
df = df.drop(['RowNumber','Surname','CustomerId'],axis = 1)

In [43]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [44]:
# converting categorical columns to numerical by  OHE
ohe  = OneHotEncoder(sparse_output=False)
encoded_features = ohe.fit_transform(df[['Geography']])

In [45]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [46]:
encoded_features_df = pd.DataFrame(encoded_features,columns=ohe.get_feature_names_out(['Geography']))

In [48]:
df = pd.concat([df.drop('Geography',axis=1),encoded_features_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [53]:
pwd

'/Users/abhi/code/Projects/ANN-project'

In [54]:
## Save the encoders and sscaler
with open('./pickle/gender_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder,file)

with open('./pickle/onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe,file)

In [55]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [57]:
X=df.drop('Exited',axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [60]:
#Saving scaler 
with open("./pickle/scaler.pkl", 'wb') as file:
    pickle.dump(scaler,file)

In [61]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [70]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],)),  ## HL1 Connected wwith input layer
    Dense(32,activation='relu'),   ## HL2
    Dense(1,activation='sigmoid')   ## output layer
])

/Users/abhi/code/Projects/ANN-project/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [64]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)  
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [66]:
# model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy']) #by default learning_rate is 0.001
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [67]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [68]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [69]:
history = model.fit(
    X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),
    epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8188 - loss: 0.4347 - val_accuracy: 0.8576 - val_loss: 0.3778
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.8483 - loss: 0.3690 - val_accuracy: 0.8600 - val_loss: 0.3574
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.8579 - loss: 0.3548 - val_accuracy: 0.8608 - val_loss: 0.3439
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8629 - loss: 0.3373 - val_accuracy: 0.8584 - val_loss: 0.3572
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8627 - loss: 0.3411 - val_accuracy: 0.8568 - val_loss: 0.3432
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.8595 - loss: 0.3401 - val_accuracy: 0.8656 - val_loss: 0.3434
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - accuracy: 0.8636 - loss: 0.3412 - val_accuracy: 0.8584 - val_loss: 0.3516
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.8785 - loss: 0.3

In [71]:
model.save('./pickle/model.h5')

In [72]:
## Load Tensorboard Extension
%load_ext tensorboard

In [74]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11321), started 0:00:14 ago. (Use '!kill 11321' to kill it.)